In [1]:
%matplotlib inline

In [2]:
cd ..

/home/andre/Projects/stocks


In [3]:
from matplotlib import pyplot as plt
import os
import sys
import pandas as pd
import numpy as np
import struct
import datetime
from Tools.util import progressbar
from Tools import prepareData, torchNN, torchCV
from Tools import meta5Ibov
import seaborn as sns


### Load Vectors Ready for Training

In [4]:
cd data

/home/andre/Projects/stocks/data


In [5]:
# # Working Path for Expert Advisor Metatrader 5
# # Use same path that can be read by the expert advisor
# #datapath = "C:\\Users\\alferreira\\Downloads\\stocks\\data"
# datapath = "/home/andre/Projects/stocks/data"
# os.chdir(datapath)
# # staistical mean and variance from 2013+2018 used to make data
# # with variance < 1 and mean close to 0, that is how it works!
# #stocks_stats = pd.read_csv('stocks_stats_2018.csv', index_col=0)
# # selected columns by backtesting best performance accuracy
# with open('collumns_selected.txt', 'r') as f: # txt column names divided by spaces
#     columns = f.read()
# selected_columns = columns.split(' ')[:-1]
# #datapath = '/home/andre/.wine/drive_c/Program Files/Rico MetaTrader 5/MQL5/Files'
# meta5Ibov.setDataPath(datapath, datapath, verbose=False)
# # don't want masterdf to be reused
# masterdf = meta5Ibov.loadMeta5Data(suffix='M1.mt5bin',
#             cleandays=True, verbose=True)
# masterdf = meta5Ibov.simpleColumnNames()

In [6]:
# percmiss = meta5Ibov.calculateMissing(masterdf)
# X, y, Xp = prepareData.GetTrainingPredictionVectors(
#     masterdf.copy(), targetsymbol='PETR4_C', verbose=False,
#     span=90, stats=None, selected=selected_columns)

In [7]:
# X.to_pickle('X_train_90_minutes.pickle')
# y.to_pickle('Y_train_90_minutes.pickle')

In [8]:
X = pd.read_pickle('X_train_90_minutes.pickle')
y = pd.read_pickle('Y_train_90_minutes.pickle')

In [9]:
# X = X[:80000]
# y = y[:80000]

In [10]:
X.shape

(504965, 148)

Clean up training samples  

1:30 hours = 90 minutes  

day has 7 hours for trading == 7*60 = 420  
420-90-90 = 240 boundaries make left = 4 hours  

In [11]:
X['day'] = X.index.map(lambda x: x.date())

In [12]:
for day, group in X.groupby('day'):
    if len(group) < 60*4: # less than 4 hours cannot daytrade with 2/3 hours boundary
        X.drop(group.index, inplace=True)

In [13]:
X['start'] = X.day.map(lambda x: datetime.datetime(x.year, x.month, x.day, 11, 30, 0))
X['end'] = X.day.map(lambda x: datetime.datetime(x.year, x.month, x.day, 15, 30, 0))
Xn = X.query('index > start and index < end').copy()
Xn.drop(columns=['day', 'start', 'end'], inplace=True)
Yn = y[Xn.index]

In [14]:
Yn.tail(3)

time
2018-07-27 15:22:00    1.0
2018-07-27 15:23:00    1.0
2018-07-27 15:24:00    1.0
Name: y, dtype: float64

In [15]:
Xn.head(3)

,BBAS3_O,BBAS3_T,BBAS3_R,DOL$_O,DOL$_T,DOL$_R,VALE3_O,VALE3_T,VALE3_R,ABEV3_O,...,dema_2hour,macdhour,macd_meanhour,ema_2weekday,dema_2weekday,macdweekday,macd_meanweekday,rsi_2weekday,ema_2minute,macdminute
time,,,,,,,,,,,,,,,,,,,,,
2013-07-02 11:31:00,-1.039078,-0.234609,-0.621252,-0.517422,1.251865,1.862783,-0.185586,-0.132574,-0.098213,-1.229889,...,-0.871298,0.169966,-1.602625,-1.148629,0.632565,0.021266,0.896495,0.723836,-1.542713,0.397759
2013-07-02 11:32:00,-1.037687,-0.030084,-0.261802,-0.515785,1.626139,1.169825,-0.185586,-0.525525,-0.583688,-1.225708,...,-0.861675,0.158380,-1.595915,-1.144886,0.625563,0.019656,0.893689,0.723836,-1.508259,0.459120
2013-07-02 11:33:00,-1.037687,0.112929,-0.352245,-0.517422,1.327723,0.199945,-0.187598,-0.525525,-0.649792,-1.229889,...,-0.852158,0.147496,-1.589038,-1.141185,0.618639,0.018165,0.890757,0.723836,-1.466666,0.516576


In [16]:
device = torchNN.getDevice()

In [17]:
device

device(type='cuda')

In [18]:
# tensors cannot be used with sklearn train_test_split
# Xth, Yth = torchNN.dfvectorstoTensor(Xn, Yn, device)
# Xth = torchNN.tensorNormalize(Xth)

In [19]:
torchNN.setSeed(device, 25640) # reproducible

In [20]:
len(Xn)

288190

Train in **1 year** and test on the subsequent **6 months**

In [21]:
#indextrain, indextest, nfolds = indexSequentialFolds(len(Xn), 4*60*5*4, 4*60)
ntrain= 4*60*5*4
ntest = 4*60
indexfolds, nfolds = torchCV.indexSequentialFolds(len(Xn), ntrain+ntest)

number folds  283150


Get a random begin on data of 5 years

In [22]:
i = np.random.randint(nfolds)

Locally normalize mean = 0 and variance to 1

In [23]:
Xfold, yfold  = Xn[indexfolds[i,0]:indexfolds[i,1]].copy(), Yn[indexfolds[i,0]:indexfolds[i,1]].copy()
Xfold = (Xfold - Xfold.mean())/Xfold.std() # normalize variance=1 mean=0

In [24]:
# tensors
Xt, yt = torchNN.dfvectorstoTensor(Xfold[:ntrain], yfold[:ntrain], device) 
Xs, ys = torchNN.dfvectorstoTensor(Xfold[-ntest:], yfold[-ntest:], device)

KeyboardInterrupt: 

In [ ]:
classifier = torchNN.BinaryNN(device, dropout=0.4, learn=5e-4, patience=7)

In [ ]:
classifier.fit(Xt, yt, Xs, ys, epochs=15, batch=64, score=0.5, gma=0.725, verbose=False)

In [ ]:
results = classifier.results()

In [ ]:
results.head()

### Best 54% on validation

- `gamma`  should not be too high (~0.9) otherwise we have overfitting on training set 
- a low `gamma` value makes it go to a local minima that is the validation minima

In [ ]:
xlim = 1200#len(errort)#40 #len(errort) #50#len(errort) #310
epochs = results.index[-xlim:]
losst = results.t_loss.values[-xlim:]
lossv = results.v_loss.values[-xlim:]
acc = results.accuracy.values[-xlim:]
f, axr = plt.subplots(3, sharex=True, figsize=(15,5))
f.subplots_adjust(hspace=0)
axr[0].plot(epochs, acc, '.r')
axr[0].plot(epochs, acc, '-r', label='accuracy validation')
#axr[0].set_xscale('log')
axr[0].grid()
axr[0].legend()
axr[1].plot(epochs, lossv, '.g')
axr[1].plot(epochs, lossv, '-g', label='loss validation')
axr[1].grid()
axr[1].legend()
axr[2].plot(epochs, losst, '.r')
axr[2].plot(epochs, losst, '-r', label='loss training')
axr[2].grid()
axr[2].legend()
axr[2].set_xlabel('epochs')

### Cross validation Sequential Folds - random samples

In [ ]:
torchNN.setSeed(device, 25640) # reproducible

In [ ]:
#def cross_validate
ntrain= 4*60*5 # week
ntest = 90
nacc = 90
indexfolds, nfolds = torchCV.indexSequentialFolds(len(Xn), ntrain+ntest+nacc)
accuracies = [] # validation accuracies
for n in progressbar(range(1000)):
    i = np.random.randint(nfolds)
    Xfold, yfold  = Xn[indexfolds[i,0]:indexfolds[i,1]].copy(), Yn[indexfolds[i,0]:indexfolds[i,1]].copy()
    Xfold = (Xfold - Xfold.mean())/Xfold.std() # normalize variance=1 mean=0
    # tensors
    Xt, yt = torchNN.dfvectorstoTensor(Xfold[:ntrain], yfold[:ntrain], device) 
    Xs, ys = torchNN.dfvectorstoTensor(Xfold[ntrain:-nacc], yfold[ntrain:-nacc], device)
    classifier = torchNN.BinaryNN(device, dropout=0.4, learn=5e-4, patience=10)
    accuracyt = classifier.fit(Xt, yt, Xs, ys, epochs=15, batch=32, score=0.5, gma=0.725, verbose=False)
#     Xt, yt = torchNN.dfvectorstoTensor(Xfold[:ntrain+ntest//2], yfold[:ntrain+ntest//2], device) 
#     Xs, ys = torchNN.dfvectorstoTensor(Xfold[ntrain+ntest//2:-nacc], yfold[ntrain+ntest//2:-nacc], device)
#     classifier = torchNN.BinaryNN(dropout=0.4, learn=5e-4, patience=10)
#     accuracyv = classifier.fit(Xt, yt, Xs, ys, device, epochs=15, batch=32, score=0.5, gma=0.725, verbose=False)
    classifier.fineTune(Xs, ys, epochs=1, batch=8) # fine tunning before predict
    Xac, yac = torchNN.dfvectorstoTensor( Xfold[-nacc:], yfold[-nacc:], device)
    predicted = torchNN.modelAccuracy(classifier, Xac, yac)
    accuracies.append([accuracyt, 0., predicted])
    print(n)

In [ ]:
accuracies = np.array(accuracies)

In [ ]:
def probability(values):
    percentiles = np.percentile(values, [0, 1, 10, 50, 90])
    print('percentiles are ', percentiles)
    import matplotlib.ticker as plticker
    loc = plticker.MultipleLocator(base=1.0) # this locator puts ticks at regular intervals
    fig = plt.figure(figsize=(10,4))
    ax1 = fig.add_subplot(211);
    ax1.hist(values, bins=20, density=True);
    #ax1.set_xlim(percentiles[0], 5)
    ax1.xaxis.set_major_locator(loc)
    ax1.grid()
    ax2 = fig.add_subplot(212, sharex=ax1)
    ax2.hist(values, bins=20, density=True, cumulative=True);
    ax2.xaxis.set_major_locator(loc)
    ax2.set_ylabel('cumulative percentage (%)')
    ax2.grid()

In [ ]:
dfacc = pd.DataFrame(accuracies, columns=['score0', 'score1', 'acc'])

In [ ]:
#dfacc = pd.read_pickle('simulations_akamoviedod.pickle')

In [ ]:
dfacc.head(1)

In [ ]:
#dfacc['avg'] = np.sqrt(dfacc.score0*dfacc.score1)

In [ ]:
plt.figure()
sns.jointplot(dfacc.score0, dfacc.acc, kind="kde")
sns.despine()
plt.figure()

In [ ]:
probability(dfacc[dfacc.score0 > 0.95].acc);

In [ ]:
len(dfacc[dfacc.score0 > 0.95].acc)

In [ ]:
len(dfacc[dfacc.score0 > 0.95].acc)/1000